In [1]:
import os
import random

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tqdm.notebook import tqdm
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, LSTM, LSTMCell, GRU, GRUCell, Reshape, Dropout, GaussianNoise, Concatenate, Lambda, RepeatVector

# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

### Yahoo S5

In [ ]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [ ]:
for loader in [load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4]:
    datasets = loader(128, 32)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
#         N_RES = 3

#         seq_length, dim = X_train.shape[1], X_train.shape[2]
#         X_train_reverse = np.flip(X_train, axis=1)
#         X_test_reverse = np.flip(X_test, axis=1)

#         X_train_by_res = []
#         X_test_by_res = []
#         X_train_by_res_reverse = []
        
#         for n in range(N_RES):
#             selected_t = [t for t in range(0, seq_length, 2**n)]
#             X_train_by_res.append(X_train[:, selected_t, :])
#             X_test_by_res.append(X_test[:, selected_t, :])
#             X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
#         model = SKip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
#         history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
#                             callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
#         X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
#         scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)
    
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

In [ ]:
yahoo_results = pd.DataFrame(total_scores)

In [ ]:
yahoo_results.groupby('dataset').mean()

### NASA

In [ ]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [ ]:
for loader in [load_nasa]:
    datasets = loader(100, 100)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        
        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

In [ ]:
nasa_results = pd.DataFrame(total_scores)

In [ ]:
nasa_results.groupby('dataset').mean()

### SMD

In [3]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [ ]:
for loader in [load_smd]:
    datasets = loader(100, 100)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

       
      
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

In [ ]:
smd_results = pd.DataFrame(total_scores)

In [ ]:
smd_results.groupby('dataset').mean()